# Olfactory Projection Fibers (OP_2)

In [1]:
from ip.ios import *
from ip.enhancement import *
from ip.binary import *
from ip.graph_nx import *
from ip.swc import *
from ip.utils import *
from skimage.morphology import skeletonize
from skimage.util import img_as_ubyte
from scipy import ndimage

## Read Data

In [2]:
folder_path = "./OlfactoryProjectionFibers/ImageStacks/OP_2"

In [3]:
images = load_tif_stack(folder_path)

## Code Process and Execution

In [4]:
bilateral = bilateral_blur(images, 11, 200, 200)

In [5]:
threshold1 = mean_threshold(bilateral, 50)
binary1 = simple_binary(bilateral, threshold1)
seg1 = segment(bilateral, binary1)

threshold2 = mean_threshold(seg1, 50)
binary2 = simple_binary(seg1, threshold2)


In [6]:
skel = img_as_ubyte(skeletonize(binary2))

In [7]:
strel = ndimage.generate_binary_structure(3, 1).astype("uint8")

In [8]:
minimum = ndimage.minimum_filter(images, size=2)
filtered = ndimage.median_filter(minimum, size=3)
bilateral = bilateral_blur(minimum, 7, 100, 20)

In [9]:
threshold2 = mean_threshold(bilateral)
binary2 = simple_binary(bilateral, threshold2)
seg2 = segment(bilateral, binary2)
threshold2 = mean_threshold(seg2)
binary2 = simple_binary(seg2, threshold2)

In [10]:
er = img_as_ubyte(ndimage.binary_erosion(binary2))
skel2 = img_as_ubyte(skeletonize(er))

In [11]:

# Supondo que sua imagem seja um array chamado 'image'
non_zero_slices = np.sum(skel2, axis=(1, 2)) != 0
# Inverter o boolean array para selecionar apenas os slices não vazios
keep_slices = ~non_zero_slices

# Usar np.delete para manter apenas os slices não vazios
result = np.delete(skel2, np.where(keep_slices), axis=0)

### For visualization purposes only

In [12]:
cv2_imshow([blended(images), blended(minimum), blended(skel2), blended(bilateral)])
#cv2_imshow(result[24:27])

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


True

### Graph generation

In [38]:
graph = Graph(result)
graph.set_root((26, 393, 0))
graph.create_graph()
root = list(graph.get_root())[::-1]
g_root = (0.72501,391.08,25)
print(f"OP_1 GOLD STANDARD ROOT: {g_root}\nTEST ROOT: {tuple(root)}")

>> Graph created
OP_1 GOLD STANDARD ROOT: (0.72501, 391.08, 25)
TEST ROOT: (0, 393, 26)


In [39]:
mst = graph.apply_dfs_and_label_nodes()

>> Minimum Spanning Tree Generated
>> Minimum Spanning Tree length: 3040
>> Depth-First search and labeling complete


In [40]:
graph.save_to_swc(mst,"./Test/OP_2test2.swc")

>> SWC saved at: ./Test/OP_2test.swc


True